## Setup


In [19]:
import os
import json
import requests

API_URL = 'https://api.app.genei.io/graphql'
API_KEY = 'sk_6c34c355499a44c382c4aa0ace62b215'
HEADERS = {
    'Content-Type': 'application/json',
    'x-api-key': API_KEY
}

data_dir_path = 'data/'
dist_dir_path = 'dist/'

## Retrieve Genei note data


### Search Genei notes


In [26]:
note_name = "1.0 - Quantitative Data Analysis - 'Introduction' - Week 1 Lecture Notes"

# Search for notes in Genei by name
query = """
    query {{
        searchByName(input: {{
            name: "{}"
        }}) {{
            results {{
                item {{
                    __typename
                    ... on Note {{
                        id
                        name
                    }}
                }}
                score
            }}
        }}
    }}
""".format(note_name)
try:
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    # Raise an exception if the response status code is not 200
    response.raise_for_status()
    search_results = response.json()['data']['searchByName']['results']
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    search_results = []

top_search_result_id = None
if len(search_results) > 0:
    top_search_result_id = search_results[0]['item']['id']

top_search_result_id

'note_cl8vi4vr3008701kw3s6ue0y7'

### Return note data from top search result


In [27]:
if top_search_result_id:
    query = """
    query {{
        note(id: "{}") {{
            data
        }}
    }}
    """.format(top_search_result_id)
    response = requests.post(API_URL, json={'query': query}, headers=HEADERS)
    note = response.json()
    note_data = note['data']['note']['data']['root']

    if not os.path.exists(data_dir_path):
        os.makedirs(data_dir_path)

    file_name = os.path.join(data_dir_path, note_name + '.json')
    with open(file_name, 'w') as f:
        json.dump(note_data, f)

## Convert Genei note data to markdown


In [28]:
def json_to_md(node):
    if node['type'] in ['text', 'code-highlight'] and node['text'] != '[link]':
        return node['text']

    if node['type'] == 'heading':
        level = int(node['tag'][1])
        return f"{'#' * level} " + ' '.join([json_to_md(child) for child in node['children']]) + "\n"

    if node['type'] == 'list':
        return '\n'.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'listitem':
        prefix = "- "
        if node.get('checked'):
            prefix = "- [x] "
        return prefix + ' '.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'code':
        return f"```{node.get('language', '')}\n" + ''.join([json_to_md(child) for child in node['children']]) + "\n```"

    if node['type'] == 'equation':
        sign = '$' if node['inline'] else '$$'
        return sign + node['equation'] + sign

    if node['type'] == 'quote':
        return '> ' + ' '.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'resource-link':
        return ' '.join([json_to_md(child) for child in node['children']])

    if node['type'] in ['paragraph', 'root']:
        return '\n\n'.join([json_to_md(child) for child in node['children']])

    if node['type'] == 'linebreak':
        return '\n'

    return ''


markdown_output = json_to_md(note_data)

if not os.path.exists(dist_dir_path):
    os.makedirs(dist_dir_path)

file_name = os.path.join(dist_dir_path, note_name + '.md')
with open(file_name, 'w') as f:
    f.write(markdown_output)

TypeError: sequence item 0: expected str instance, list found